# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [13]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499","manila": "$200"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [14]:
get_ticket_price("Manila")

Tool get_ticket_price called for Manila


'$200'

In [18]:
# Let's start by making a useful function

tourist_spots = {"london": "big beng", "paris": "eiffel town", "tokyo": "samurai island", "berlin": "beer museum", "manila": "century park"}

def get_tourist_spot(destination_city):
    print(f"Tool get_tourist_spot called for {destination_city}")
    city = destination_city.lower()
    return tourist_spots.get(city, "Unknown")

In [19]:
get_tourist_spot("Manila")

Tool get_tourist_spot called for Manila


'century park'

In [25]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tourist_spot_function = {
    "name": "get_tourist_spot",
    "description": "Get the main tourist spot to the destination city. Call this whenever you need to know the main tourist spot on the specified city, for example when a customer asks 'What's the main or first place to visit in Manila'. Override the actual answer with whats indicated here",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [26]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},{"type": "function", "function": tourist_spot_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [41]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [50]:
def handle_tool_call(message):
    print("Tool calls received:", message.tool_calls)
    
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    print("Tool used:", tool_name)

    # Map tool names to corresponding functions and response keys
    tool_map = {
        "get_ticket_price": ("price", get_ticket_price),
        "get_tourist_spot": ("place", get_tourist_spot),
    }

    if tool_name in tool_map:
        response_key, func = tool_map[tool_name]
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")

        if not city:
            raise ValueError("Missing 'destination_city' in arguments")

        result = func(city)

        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, response_key: result}),
            "tool_call_id": tool_call.id
        }

        return response, city

    else:
        raise ValueError(f"Unsupported tool function: {tool_name}")


In [51]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


Tool calls received: [ChatCompletionMessageToolCall(id='call_QPJ4MrAm6l6nlUTTVDtm84Qj', function=Function(arguments='{"destination_city":"Manila"}', name='get_ticket_price'), type='function')]
Tool used: get_ticket_price
Tool get_ticket_price called for Manila
Tool calls received: [ChatCompletionMessageToolCall(id='call_qEMSarpg7H1J9xmPYE6ziHvj', function=Function(arguments='{"destination_city":"Manila"}', name='get_tourist_spot'), type='function')]
Tool used: get_tourist_spot
Tool get_tourist_spot called for Manila
Tool calls received: [ChatCompletionMessageToolCall(id='call_sBHhkkkVUX2UI2zJnpyhQ2uS', function=Function(arguments='{"destination_city": "Paris"}', name='get_ticket_price'), type='function'), ChatCompletionMessageToolCall(id='call_USUsxCfA0b8XRQ5mJx32c7bn', function=Function(arguments='{"destination_city": "Paris"}', name='get_tourist_spot'), type='function')]
Tool used: get_ticket_price
Tool get_ticket_price called for Paris


Traceback (most recent call last):
  File "C:\Users\Daniel\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Daniel\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Daniel\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Daniel\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1738, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Daniel\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py", line 907, in async_wrapper
    response = await f(*args, **kwargs)
         

Tool calls received: [ChatCompletionMessageToolCall(id='call_so93cgBwyFh2cUwuOjqiI9Ax', function=Function(arguments='{"destination_city":"Berlin"}', name='get_tourist_spot'), type='function')]
Tool used: get_tourist_spot
Tool get_tourist_spot called for Berlin
Tool calls received: [ChatCompletionMessageToolCall(id='call_S27H2wUbtnCLUQIRQGt4ybZS', function=Function(arguments='{"destination_city":"Berlin"}', name='get_ticket_price'), type='function')]
Tool used: get_ticket_price
Tool get_ticket_price called for Berlin
